In [45]:
# Stock Market Prediciton Bot
# Version 1.0
# Developed by Jacob Townsend
# 04-20-2022

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import yfinance as yf
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('bmh')

In [55]:
def btn_eventhandler(obj):
    with outs:
        # Start by clearing our output just in case this isn't the first time the button was clicked
        outs.clear_output()
        
        # Try to look up the user-provided ticker, print an error and return if there's a problem
        # If we find the ticker, download it's pricing data and store it in a dataframe
        title = userin.value
        df=None
        try:
            print('Downloading ticker...')
            df = yf.download(title)
        except:
            print('Please try looking up a different ticker')
            return
        
        # Here we will print our first data visual, a table of the values we will be analyzing
        display(df)

        
        #Now we will modify our dataframe and drop any data points we don't need
        df = df[['Close']]
        future_days = 25
        df['Prediction'] = df[['Close']].shift(-future_days)
        X = np.array(df.drop(['Prediction'], 1))[:-future_days]
        y = np.array(df['Prediction'])[:-future_days]

        #Now we can split our data into training and testing groups and train the model
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
        tree = DecisionTreeRegressor().fit(x_train, y_train)
        
        #Here we are modifying our input and running our model
        x_future = df.drop(['Prediction'], 1)[:-future_days]
        x_future = x_future.tail(future_days) 
        x_future = np.array(x_future)
        predictions = tree.predict(x_future)

    
        #Print data points we can use to evaluate the accuracy of our model...
        print("Our model r2 score (coefficient of determination) is: ", (tree.score(x_test, y_test)))
    
        #create a scatter plot of our predictions
        plt.figure(figsize=(16,8))
        plt.xlabel('Time',fontsize=18)
        plt.ylabel('Predicted Close Price ($USD)',fontsize=18)
        plt.scatter(df.index.values[-25:], predictions[-25:])
        plt.title(title + ' - Scatter plot of predictions')
        plt.show()
        
        #creat a line graph of our results
        valid =  df[X.shape[0]:]
        valid['Predictions'] = predictions
        plt.figure(figsize=(16,8))
        plt.title(title + ' - Predictions')
        plt.xlabel('Time',fontsize=18)
        plt.ylabel('Close Price ($USD)',fontsize=18)
        plt.plot(df['Close'])
        plt.plot(valid[['Close','Predictions']])
        plt.legend(['Value', 'Actual Close', 'Prediction' ], loc='lower right')
        plt.show()

        
        
#Creating and displaying widgets we will use in our dashboard
sub = widgets.Button(description='Go!')
userin = widgets.Text(value='AMZN')
sub.on_click(btn_eventhandler)
outs = widgets.Output()
    
print("Welcome to the stock predictor!")
print("Just input any stock or cryptocurrecny ticker and click 'Go!'")
display(userin)
display(sub)
display(outs)



Welcome to the stock predictor!
Just input any stock or cryptocurrecny ticker and click 'Go!'


Text(value='AMZN')

Button(description='Go!', style=ButtonStyle())

Output()